In [ ]:
import os 
os.chdir('../..')

In [ ]:
from openai import OpenAI
import base64
import os

BOSON_API_KEY = "bai-Diz6JrS6rquzG1HSby-07fYX0AEgNJrCXKx0n6qr8F06ACSz"
client = OpenAI(api_key=BOSON_API_KEY, base_url="https://hackathon.boson.ai/v1")

def b64(path):
    return base64.b64encode(open(path, "rb").read()).decode("utf-8")

reference_path = "/Users/vishnou/Documents/echo-charlie/data/audio/chaplin_voice.wav"
reference_transcript = (
    "But we have lost the way. Greed has poisoned men’s souls, has barricaded the world with hate, has goose-stepped us into misery and bloodshed."
)

In [ ]:
prompt_1 = """Echo Charlie is a multimodal voice conversion system that takes a muted video and transforms it into a fully voiced performance.

Here’s how it works.
When you upload a video in the Streamlit app, Echo Charlie first samples a few key frames and extracts visual embeddings from each one. This helps identify who’s speaking and what they look like.

Next, it searches the internal EchoDB — a ChromaDB-powered library of indexed voices — to find the closest matching reference audio. This ensures the generated speech sounds like the right person.

Then, the lip-reading module steps in. Using the pre-trained LRS3 model, it decodes the silent mouth movements into raw text. That transcript is then cleaned up and punctuated by the Boson-hosted Qwen-32B large language model."""

In [ ]:
prompt_2 = '''
After that, the Higgs speech model takes over. It combines the polished transcript with the retrieved speaker's voice from the vector database to generate a realistic voice track that matches the target speaker’s tone and rhythm.

Finally, the Wav2Lip GAN model synchronizes it all — fusing the cloned audio with the original video to produce a perfectly lip-synced result. The silent clip now speaks naturally, as if it were never muted.
'''

In [ ]:
prompt_3 = """Echo Charlie can be a powerful tool for film restoration and post-production — when audio tracks are lost, damaged, or need to be re-recorded without bringing actors back to the studio.
It can also help recover archival footage, documentaries, or interviews where sound wasn’t captured properly, seamlessly restoring speech to match the original visuals.
Beyond that, it opens creative possibilities for dubbing, accessibility, and automated voice recreation across languages and styles."""

In [ ]:
prompt_4 = """In the Streamlit demo, you can explore every stage:
Upload a muted video,
Watch the lip-reading output,
Review the cleaned transcript,
Listen to the generated voice,
And preview the final synced video.
Echo Charlie — turning silence into seamless conversation."""

In [ ]:
resp = client.chat.completions.create(
    model="higgs-audio-generation-Hackathon",
    messages=[
        {"role": "user", "content": reference_transcript},
        {
            "role": "assistant",
            "content": [{
                "type": "input_audio",
                "input_audio": {"data": b64(reference_path), "format": "wav"}
            }],
        },
        {"role": "user", "content": prompt_4},
    ],
    modalities=["text", "audio"],
    max_completion_tokens=4096,
    temperature=1.0,
    top_p=0.95,
    stream=False,
    stop=["<|eot_id|>", "<|end_of_text|>", "<|audio_eos|>"],
    extra_body={"top_k": 50},
)

audio_b64 = resp.choices[0].message.audio.data
open("chaplin_narrated_3.wav", "wb").write(base64.b64decode(audio_b64))